## **ENCRYPTOR**

In [2]:
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import string
import random
import secrets

class EncryptionStrategy:
    def encrypt(self, message):
        pass

    def decrypt(self, encrypted_message):
        pass

class AESCipher(EncryptionStrategy):
    def __init__(self, key):
        self._key = key

    def encrypt(self, message):
        iv = secrets.token_bytes(16)
        cipher = Cipher(algorithms.AES(self._key), modes.CBC(iv))
        encryptor = cipher.encryptor()

        # Encode the message as bytes
        message_bytes = message.encode('utf-8')

        # Apply padding to the message
        padder = padding.PKCS7(128).padder()
        padded_message = padder.update(message_bytes) + padder.finalize()

        ct = encryptor.update(padded_message) + encryptor.finalize()
        return iv + ct

    def decrypt(self, encrypted_message):
        iv = encrypted_message[:16]
        ct = encrypted_message[16:]
        cipher = Cipher(algorithms.AES(self._key), modes.CBC(iv))
        decryptor = cipher.decryptor()

        # Decrypt the ciphertext
        pt = decryptor.update(ct) + decryptor.finalize()

        # Create a new unpadder using the original padding algorithm
        unpadder = padding.PKCS7(128).unpadder()

        # Update the unpadder with the decrypted bytes
        unpadded_message_bytes = unpadder.update(pt)

        # Finalize the unpadder to remove padding
        try:
            unpadded_message_bytes += unpadder.finalize()
        except ValueError:
            raise ValueError("Invalid padding bytes.")

        return unpadded_message_bytes.decode('utf-8')

class Encryptor:
    def __init__(self, strategy):
        self._strategy = strategy

    def set_key(self, key):
        self._key = key

    def encrypt_message(self, message):
        return self._strategy.encrypt(message)

    def decrypt_message(self, encrypted_message):
        return self._strategy.decrypt(encrypted_message)

class EncryptionComplexity:
    def __init__(self):
        self._strategies = {}

    def register_strategy(self, complexity, strategy):
        self._strategies[complexity] = strategy

    def get_strategy(self, complexity):
        return self._strategies.get(complexity)

    def _generate_token(self, encrypted_message):
        characters = string.ascii_uppercase + string.digits
        token = ''.join(random.choice(characters) for _ in range(len(encrypted_message)))
        return token

    def _parse_token(self, token):
        # Convert the token back to bytes
        token_bytes = token.encode('utf-8')

        # Extract the IV (first 16 bytes of the token)
        iv = token_bytes[:16]

        # Extract the encrypted message and pad length from the token
        encrypted_message = token_bytes[16:-1]
        pad_length = ord(token[-1])

        return encrypted_message, pad_length

    def encrypt_message(self, message, complexity):
        strategy = self.get_strategy(complexity)
        if strategy:
            encrypted_message = strategy.encrypt(message)
            token = self._generate_token(encrypted_message)
            return token
        else:
            raise ValueError(f"Encryption complexity level '{complexity}' not found.")

    def decrypt_message(self, token, complexity):
        strategy = self.get_strategy(complexity)
        if strategy:
            encrypted_message, pad_length = self._parse_token(token)
            decrypted_message = strategy.decrypt(encrypted_message)
            return decrypted_message[:-pad_length]
        else:
            raise ValueError(f"Encryption complexity level '{complexity}' not found.")

if __name__ == "__main__":
    # Create an instance of the EncryptionComplexity class
    encryption_complexity = EncryptionComplexity()

    # Generate encryption keys for different complexity levels (Low, Medium, High)
    key_low = secrets.token_bytes(16)    # 16 bytes (128 bits) key
    key_medium = secrets.token_bytes(24) # 24 bytes (192 bits) key
    key_high = secrets.token_bytes(32)   # 32 bytes (256 bits) key

    # Register AESCipher strategies with their corresponding encryption keys for each complexity level
    encryption_complexity.register_strategy("Low", AESCipher(key_low))
    encryption_complexity.register_strategy("Medium", AESCipher(key_medium))
    encryption_complexity.register_strategy("High", AESCipher(key_high))

    # Message to be encrypted
    message = "This is a secret message."

    # Encrypt the message using the desired complexity level to obtain a token-like encrypted result
    token_low = encryption_complexity.encrypt_message(message, "Low")
    token_medium = encryption_complexity.encrypt_message(message, "Medium")
    token_high = encryption_complexity.encrypt_message(message, "High")

    # Display the encrypted message in token-like format
    print("Message:", message)
    print("Token (Low complexity):", token_low)
    print("Token (Medium complexity):", token_medium)
    print("Token (High complexity):", token_high)


Message: This is a secret message.
Token (Low complexity): N07DRANU32Q2VTWY4OUCFU2J6KSRVU1NZNYRNA0CCDW94BSZ
Token (Medium complexity): LACCEUTFMOTO35NP2RL5GYKBV9N045F0AZBT38OHBWQRRUEC
Token (High complexity): H6MJXB6KJHJVI6LZEJUIKN9OWRPR3K1DK1X6ITNJ8E58V2XW


## **ON PROGRESS**

In [ ]:
    # # Decrypt the token-like encrypted message back to the original message using the appropriate complexity level
    # decrypted_message_low = encryption_complexity.decrypt_message(token_low, "Low")
    # decrypted_message_medium = encryption_complexity.decrypt_message(token_medium, "Medium")
    # decrypted_message_high = encryption_complexity.decrypt_message(token_high, "High")

    # # # Display the decrypted messages
    # # print("\nDecrypted message (Low complexity):", decrypted_message_low)
    # # print("Decrypted message (Medium complexity):", decrypted_message_medium)
    # # print("Decrypted message (High complexity):", decrypted_message_high)
